# Homework: Decipherment

In [1]:
%load_ext autoreload
%autoreload 2
from collections import defaultdict, Counter
import collections
import pprint
import math
import bz2
from ngram import *
import sys, string
import copy
import pickle
#from joblib import Parallel, delayed
import itertools
pp = pprint.PrettyPrinter(width=45, compact=True)

In [2]:
import numpy as np

First let us read in the cipher text from the `data` directory:

In [3]:
def read_file(filename):
    if filename[-4:] == ".bz2":
        with bz2.open(filename, 'rt', encoding='utf8') as f:
            content = f.read()
            f.close()
    else:
        with open(filename, 'r', encoding='utf8') as f:
            content = f.read()
            f.close()
    return content

cipher = read_file("data/cipher.txt")
print(cipher)

º∫P/Z/uB∫ÀOR•–X•B
WV+≈GyF∞ºHPπKÇ—y≈
MJy^uIÀΩ—T‘NQyDµ£
S¢/º∑BPORAu∫∆RÃ—E
À^LMZJƒ“\–FHVW≈æy
π+—GDºKI£∞—Xæµ§S¢
RN‘IyEÃOæ—GBTQS∑B
Lƒ/P∑BπX—EHMu^RRÀ
√ZK—–I£W—ÇæµLM“º∑
BPDR+j•∞\N¢≈EuHÀF
Z√–OVWIµ+‘L£Ã^R∞H
IºDR∏Ty“\ƒ≈/πXJQA
PµMæRu‘∫L£NVEKH•G
“IÇJÀµºæLMÃNA£Z¢P
§u–ÀAº∑BVW\+VT‘OP
^•S“Ã∆u≈∞ΩD§G∫∫IM
NÀ£S√E/º∫∫Z∆AP∑BV
–≈X—W—∏F∑æ√+πºAºB
∫OTµRu√+∏ƒy—∏^S—W
VZ≈GyKE∏TyAº∫∑L‘∏
HÇFBXº§XADƒ\ΩLÇ•—
∏≈ƒ∑∑∞≈µPORXQF∫G√
ZπJT‘—∏æJI+“BPQW∞
VEX“ºWI∞—EHM£•uIÀ


For the default solution we need to compute statistics like length, number of symbols/letters, 
unique occurences, frequencies and relative frequencies of a given file. This is done in the function `get_statistics` below.

While using `get_statistics`, make sure that `cipher=True` is set when the input is a ciphertext.

In [4]:
def get_statistics(content, cipher=True):
    stats = {}
    content = list(content)
    split_content = [x for x in content if x != '\n' and x!=' ']
    length = len(split_content)
    symbols = set(split_content)
    uniq_sym = len(list(symbols))
    freq = collections.Counter(split_content)
    rel_freq = {}
    for sym, frequency in freq.items():
        rel_freq[sym] = (frequency/length)*100
        
    if cipher:
        stats = {'content':split_content, 'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    else:
        stats = {'length':length, 'vocab':list(symbols), 'vocab_length':uniq_sym, 'frequencies':freq, 'relative_freq':rel_freq}
    return stats

In [5]:
"""
ATTENTION!
For grading purposes only. Don't bundle with the assignment. 
Make sure '_ref.txt' is removed from the 'data' directory before publishing.
"""

def read_gold(gold_file):
    with open(gold_file) as f:
        gold = f.read()
    f.close()
    gold = list(gold.strip())
    return gold

def symbol_error_rate(dec, _gold):
    gold = read_gold(_gold)
    correct = 0
    if len(gold) == len(dec):
        for (d,g) in zip(dec, gold):
            if d==g:
                correct += 1
    wrong = len(gold)-correct
    error = wrong/len(gold)
    
    return error

In [6]:
cipher_desc = get_statistics(cipher, cipher=True)
#pp.pprint(cipher_desc)

## Load the 6-gram model

In [7]:
%%time
sequence = 'In a few cases, a multilingual artifact has been necessary to facilitate decipherment, the Rosetta Stone being the classic example. Statistical techniques provide another pathway to decipherment, as does the analysis of modern languages derived from ancient languages in which undeciphered texts are written. Archaeological and historical information is helpful in verifying hypothesized decipherments.'

# lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=True)
lm = LM("data/6-gram-wiki-char.lm.bz2", n=6, verbose=False)

Reading language model from data/6-gram-wiki-char.lm.bz2...


CPU times: user 31.3 s, sys: 803 ms, total: 32.1 s
Wall time: 32.2 s


Done.


In [8]:
print(sequence)
lm_logprob = lm.score_seq(sequence)
print("TOTAL LM LOGPROB: {}".format(lm_logprob), file=sys.stderr)
print("TOTAL LM LOGPROB: {}".format(lm.score_seq('this is the text.')), file=sys.stderr)
print("TOTAL LM LOGPROB: {}".format(lm.score_seq('jasbklfhthejkldhf')), file=sys.stderr)
print(lm.get_bitstring_spans('..oo...ooo..'))
print(lm.score_bitstring('thisisatest', 'oo...oo.ooo'))

In a few cases, a multilingual artifact has been necessary to facilitate decipherment, the Rosetta Stone being the classic example. Statistical techniques provide another pathway to decipherment, as does the analysis of modern languages derived from ancient languages in which undeciphered texts are written. Archaeological and historical information is helpful in verifying hypothesized decipherments.
{2: 3, 3: 4, 7: 8, 8: 9, 9: 10}
-8.10905897


TOTAL LM LOGPROB: -221.09434842188
TOTAL LM LOGPROB: -9.76947916
TOTAL LM LOGPROB: -40.57683077


# From Yabin: new score function (should be more efficient) 

In [35]:
def score_new(cipher, phi, new_f, new_e, previous_score):
    '''
    scores the phi_prime based on the previous score, returns a dict
    cipher -- list of single character string
    phi -- dictionary, old mapping e->[f]
    new_f -- single-character string, extended symbol
    previous_score -- float, old score for phi
    '''
    mapping = phi
    new_score = previous_score
    lm_state = lm.begin()
    for i in range(len(cipher)):
        char = cipher[i]
        if char in mapping.keys():
            token = mapping[char]
            ngram = lm_state + (token,)
            while len(ngram)> 0:
                if ngram in lm.table:
                    lm_state = ngram[-lm.history:]
                    break
                else: #backoff
                    ngram = ngram[1:]
            if len(ngram)==0:
                lm_state = ()
        elif char == new_f:
            (lm_state, logprob) = lm.score(lm_state, new_e)
            new_score += logprob
        else:
            lm_state = ()  
    return new_score

In [36]:
def beam_search_new(cipher, ext_order, ext_limits=1, topn=1):
    '''
    finds the mappings between cipher char and plaintext char, returns the mapping dictionary
    ext_order -- list, the unigram char list sorted by their count DESC
    ext_limits -- int, defines maximum number of cipher char can be mapped to a plaintext char
    topn -- int, defines the number of dictionaries we want to keep while pruning
    '''
    print('Number of unique symbols in cipher:', len(ext_order))
    # mapping relationships already found with score
    Hs = [(defaultdict(dict), 0)]
    # hypothesis mapping relationships with score
    Ht = []
    # initialize the cardinality (number of unique cipher text)
    cardinality = 0
    # list of plaintext characters
    Ve = [chr(i) for i in range(97, 123, 1)]
    while cardinality < len(ext_order):
    #while cardinality < 2:
        f = ext_order[cardinality]
        print('Working on symbol: ', f, f'({cardinality+1})')
        for phi, previous_score in Hs:
            for e in Ve:
                phi_prime = copy.deepcopy(phi)
                new_map = {f: e}
                phi_prime.update(new_map)
                counts = len([v for k, v in phi_prime.items() if v == e])
                if counts <= ext_limits:
                    Ht.append((phi_prime, score_new(cipher, phi, f, e, previous_score)))
        # prune the histogram
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]                    
        cardinality += 1
        Hs = copy.deepcopy(Ht)
        Ht.clear()
        print('Current score: ', Hs[0][1])
        #print('Hs', Hs)
    return sorted(Hs, key=lambda x:x[1], reverse=True)

# new extension order

In [9]:
def find(string, char):
    return [i for i, letter in enumerate(string) if letter == char]

In [10]:
def gen_bit_str(cipher, deciphered_symbols):
    # generate bit string
    bit_string = ''
    for char in cipher:   
        if (char in deciphered_symbols):
            bit_string += 'x'
        else: 
            bit_string += '.'
    return bit_string

In [22]:
def case1(ngram):
    #print("case 1")
    num_pos = np.zeros(ngram)
    num_pos[0] += 1
    return num_pos
def case2(ngram, str_after):
    #print("case 2")
    num_pos = np.zeros(ngram)
    if str_after.find('.') != -1:
        str_after = str_after[:str_after.find('.')]
    #print("after1: "+str_after)
    for i in range(len(str_after)):
        idx = i+1
        num_pos[idx] += 1
        if idx+1 == ngram:
            break    
    return num_pos
def case3(ngram, str_before):
    #print("case 3")
    num_pos = np.zeros(ngram)
    str_before = str_before[str_before.rfind('.')+1:]
    #print("before1: "+str_before)
    for i in range(len(str_before)):
        idx = i+1
        num_pos[idx] += 1
        if idx+1 == ngram:
            break
    return num_pos
def case4(ngram, str_before, str_after):  
    #print("case 4")
    num_pos = np.zeros(ngram)
    str_before = str_before[str_before.rfind('.')+1:]
    if str_after.find('.') != -1:
        str_after = str_after[:str_after.find('.')]
    #print("before1: "+str_before)
    #print("after1: "+str_after)
    num_pos[0] -= 1
    for i in range(len(str_before)):
        idx = i+1
        num_pos[idx] += 1
        if idx+1 == ngram:
            break    
    for i in range(len(str_after)):
        idx = i+1
        num_pos[idx] += 1
        if idx+1 == ngram:
            break    
            
    # ********* to be complete ********* #
    
    return num_pos

In [28]:
def find_ext_order(cipher, topn=100, weights=(1.0, 1.0, 1.0, 1.0, 2.0, 3.0), ngram = 6):

    # initial
    cipher_desc = get_statistics(cipher, cipher=True)
    weights = np.array(weights)
    cipher_vocab = cipher_desc['vocab']

    first_symbol = cipher_desc['frequencies'].most_common(1)[0][0] # use the most frequent symbol as first symbol
    
    cardinality = 1
    
    # symbols already found with score
    Hs = [([first_symbol], 0)]
    # hypothesis extended symbols with score
    Ht = []
    
    print('Done with symbol number', cardinality, '; Current best score: ', Hs[0][1])
    
    while cardinality < cipher_desc['vocab_length']:
        for phi, previous_score in Hs:
            
            deciphered_symbols = phi
            bit_string = gen_bit_str(cipher, deciphered_symbols)
            
            for voc in cipher_vocab:
                phi_prime = copy.deepcopy(phi)
                if voc in phi_prime:
                    continue
                else:
                    weighted_sum = 0.0
                    num_pos = np.zeros(ngram)
                    # for every occurance, 
                    for pos in find(cipher, voc):
                        str_before = bit_string[0:pos]
                        str_after = bit_string[pos+1:]
                        #print("before: "+str_before) 
                        #print("after: "+str_after) 

                        if len(str_before) == 0:
                            if str_after[0] == '.':
                                num_pos += case1(ngram)
                            else:
                                num_pos += case2(ngram, str_after)
                        elif len(str_after) == 0:
                            if str_before[-1] == '.':
                                num_pos += case1(ngram)
                            else:
                                num_pos += case3(ngram, str_before)
                        elif (str_before[-1] == '.') and (str_after[0] == '.'):
                            num_pos += case1(ngram)
                        elif (str_before[-1] == '.') and (str_after[0] == 'x'):
                            num_pos += case2(ngram, str_after)
                        elif (str_before[-1] == 'x') and (str_after[0] == '.'):
                            num_pos += case3(ngram, str_before)
                        else:
                            num_pos += case4(ngram, str_before, str_after)

                    weighted_sum += np.sum(np.multiply(weights, num_pos))
                    #print("weighted_sum: " + str(weighted_sum))
                    
                    phi_prime.append(voc)
                    this_score = previous_score + weighted_sum
                    Ht.append((phi_prime, this_score))
        
        # prune the histogram
        Ht = sorted(Ht, key=lambda x:x[1], reverse=True)[:topn]
        cardinality += 1
        Hs = copy.deepcopy(Ht)
        Ht.clear()
        
        print('Done with symbol number', cardinality, '; Current best score: ', Hs[0][1], '; Current worst score: ', Hs[-1][1])
    return sorted(Hs, key=lambda x:x[1], reverse=True)

In [46]:
%%time
# test the function above
ext_order = find_ext_order(cipher, 1000)

Done with symbol number 1 ; Current best score:  0
Done with symbol number 2 ; Current best score:  14.0 ; Current worst score:  1.0
Done with symbol number 3 ; Current best score:  26.0 ; Current worst score:  16.0
Done with symbol number 4 ; Current best score:  38.0 ; Current worst score:  33.0
Done with symbol number 5 ; Current best score:  53.0 ; Current worst score:  47.0
Done with symbol number 6 ; Current best score:  67.0 ; Current worst score:  61.0
Done with symbol number 7 ; Current best score:  84.0 ; Current worst score:  77.0
Done with symbol number 8 ; Current best score:  104.0 ; Current worst score:  95.0
Done with symbol number 9 ; Current best score:  121.0 ; Current worst score:  113.0
Done with symbol number 10 ; Current best score:  141.0 ; Current worst score:  132.0
Done with symbol number 11 ; Current best score:  162.0 ; Current worst score:  154.0
Done with symbol number 12 ; Current best score:  186.0 ; Current worst score:  176.0
Done with symbol number 1

In [ ]:
%%time
sorted_keys = sorted(cipher_desc['frequencies'], key=cipher_desc['frequencies'].get, reverse=True)
#reversed_mappings = beam_search(cipher_desc['content'], sorted_keys, 3, 100)
#reversed_mappings = beam_search(cipher_desc['content'], right_most_cipher_vocab, 8, 1000)
mappings = beam_search_new(cipher_desc['content'], ext_order[0][0], 8, 10000)

In [ ]:
mapping = mappings[0][0]

In [ ]:
decipher_text = ''
for char in cipher_desc['content']:
    decipher_text += mapping[char]
print(decipher_text)
print('score', lm.score_seq(decipher_text))
print(len(decipher_text))

In [ ]:
with open('data/_ref_Zodiac_408.txt', 'r') as fh:
    ground_truth = fh.read()
print(ground_truth)
print('score', lm.score_seq(ground_truth))
print(len(ground_truth))

In [ ]:
symbol_error_rate(decipher_text, 'data/_ref_Zodiac_408.txt')